In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install phik
!pip install prince
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report,roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin,BaseEstimator
import phik
from phik import resources, report
import prince

## Visit the [dashboard](https://stroke-risk-factors.herokuapp.com) allowing you to interactively explore the stroke risk factors 

## Introduction

***

In what follows, I will analyze the [Stroke Prediction Dataset](https://www.kaggle.com/fedesoriano/stroke-prediction-dataset) containing relevant information about ~5k patients. The aim of this study is to check whether, given a set of features describing the health status of the patient, it is possible to predict a stroke event.   

The complete list of attributes includes 12 features which are listed below:

1. id: unique identifier
2. gender: "Male", "Female" or "Other"
3. age: age of the patient
4. hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5. heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6. ever_married: "No" or "Yes"
7. work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8. Residence_type: "Rural" or "Urban"
9. avg_glucose_level: average glucose level in blood
10. bmi: body mass index
11. smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
12. stroke: 1 if the patient had a stroke or 0 if not

***

## Index

The notebook has been structured as follows:

1. [Read the dataset and get familiar with the data](#1.-Read-the-dataset-and-get-familiar-with-the-data)
    1. [Feature engineering](#1.A-Feature-engineering)
2. [Data visualization](#2.-Data-visualization)
    1. [Distributions](#2.1-Distributions)
    2. [Study of the correlation matrix](#2.2-Study-of-the-correlation-matrix)
3. [Testing resampling techniques with XGBoost binary classifier](#3.-Testing-resampling-techniques-with-XGBoost-binary-classifier)
    1. [Hyperparameters tuning with GridSearchCV](#3.1-Hyperparameters-tuning-with-GridSearchCV)
4. [Dimensionality reduction](#4.-Dimensionality-reduction)
    1. [Univariate analysis](#4.A-Univariate-analysis)
    2. <a href="#4.B-Recursive-feature-elimination-(RFE)-with-random-forestt">Recursive feature elimination (RFE) with random forest</a>
    3. <a href="#4.C-RFE-with-Cross-Validation-(RFECV)-with-random-forest">RFE with Cross Validation (RFECV) with xgboost</a>
    4. <a href="#4.D-Factor-Analysis-of-Mixed-Data-(FAMD)-for-feature-extraction">Factor Analysis of Mixed Data (FAMD) for feature extraction</a>

## 1. Read the dataset and get familiar with the data
### 1.A Feature engineering
The dataset includes the data for 5110 patients, each one being associated with a unique ID which is not a relevant feature for the analysis presented here. There are 8 categorical features:   `gender`,`hypertension`,`heart_disease`,`ever_married`,`work_type`,`residence_type`,`smoking status`,`stroke`,  
and 3 non-categorical (discrete or continuous) features  
`age`,`avg_glucose_level` and `bmi`.

The `stroke` feature will be the target variable we aim to predict. 

In [ ]:
df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()

#### Remove outliers
The gender section can take three different values: 'Male', 'Female' and 'Other'. 
However, in the next line I check that there is just one patient in the category 'Other'. For this reson, since there is not a sufficiently high number of representatives in this category to perform a statistical analysis, I will drop the corresponding line. 

In [ ]:
cond = df['gender']=='Other'
occu = len(df['gender'][cond])
print('Number of occurrences %d corresponding to %.2f%% of the dataset'%(occu, 100*(np.float64(occu)/len(df))))

Analogously, I will remove the rows corresponding to BMI>47, which are 117, with just 3 stroke occurencies.

In [ ]:
cond = (df['bmi']>47) & (df['stroke']==1)
occu = len(df['bmi'][cond])
print('Number of occurrences of BMI>47 and stroke=1 %d corresponding to %.2f%% of the dataset'%(occu, 100*(np.float64(occu)/len(df))))

cond = (df['bmi']>47) & (df['stroke']==0)
occu = len(df['bmi'][cond])
print('Number of occurrences of BMI>47 and stroke=0 %d corresponding to %.2f%% of the dataset'%(occu, 100*(np.float64(occu)/len(df))))

The same apply to average glucose level < 57, with 81 rows and just 1 occurrence of stroke.

In [ ]:
cond = (df['avg_glucose_level']<57) & (df['stroke']==1)
occu = len(df['avg_glucose_level'][cond])
print('Number of occurrences of average glucose level <57 and stroke=1 %d corresponding to %.2f%% of the dataset'%(occu, 100*(np.float64(occu)/len(df))))

cond = (df['avg_glucose_level']<57) & (df['stroke']==0)
occu = len(df['avg_glucose_level'][cond])
print('Number of occurrences of average glucose level <57 and stroke=0 %d corresponding to %.2f%% of the dataset'%(occu, 100*(np.float64(occu)/len(df))))

In [ ]:
df.isnull().sum()

There are 201 NaNs values in the bmi field, for this reason I will employ the [nearest neighbors](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer) imputation provided by the scikit-learn library.

In [ ]:
# from sklearn.impute import KNNImputer
# imp = KNNImputer(missing_values=np.NaN)
# df.bmi = imp.fit_transform(df['bmi'].values.reshape(-1,1))[:,0]

Encoding the categorical variables with the [sklearn ordinal encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder) for computing the correlation between features. However, given that there is no intrinsic order among the categories in each feature, [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) is more suitable for this task.

Many other algorithms can be tested. In particular, there is [a very interesting package](http://contrib.scikit-learn.org/category_encoders/index.html#) including many encoding algorithms.


In [ ]:
# categ_feat = ['gender','Residence_type','work_type','ever_married','smoking_status']
# encoder = OrdinalEncoder()
# result = encoder.fit_transform(df[categ_feat])
# df_ordinal = df.copy()
# df_ordinal[categ_feat] = result


# dummies = pd.get_dummies(df[categ_feat],drop_first=True )
# df_tmp = df.drop(categ_feat, axis = 1)
# df_one_hot = pd.concat([df_tmp,dummies],axis=1)
# one_hot_columns = df_one_hot.columns

### Build a pipeline performing the preprocessing steps

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.pipeline import FeatureUnion

df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
categ_feat = ['gender','Residence_type','work_type','ever_married','smoking_status','heart_disease','stroke','hypertension']
num_feat = list(set(df.columns)-set(categ_feat))
num_feat.remove('id')

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

class RemoveOutliers(BaseEstimator,TransformerMixin):
    def __init__(self,):
        pass
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        tmp = X.drop(X[X['gender']=='Other'].index)
        tmp = tmp.drop(tmp[tmp['avg_glucose_level']<57].index)
        tmp = tmp.drop(tmp[tmp['bmi']>47].index)
        return tmp

class ReturnDataFrame(BaseEstimator,TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        return pd.DataFrame(X,columns=self.attribute_names)

class OneHot(BaseEstimator,TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        dummies = pd.get_dummies(X,drop_first=True )
        df_tmp = X.drop(self.attribute_names, axis = 1)
        return pd.concat([df_tmp,dummies],axis=1) 


num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_feat)),
    ('imputer',KNNImputer(missing_values=np.NaN)),
    ('std_scaler',StandardScaler()),
    ('get_df',ReturnDataFrame(num_feat))
    ])


cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(categ_feat)),
    ('encoder',OneHot(categ_feat))
])



class MergeFeatures(BaseEstimator,TransformerMixin):
    def __init__(self, pip1,pip2):
        self.pip1 = pip1
        self.pip2 = pip2
    def fit(self, X, y=None):
        return self
    def transform(self,X,y=None):
        tmp1 =  self.pip1.fit_transform(X)
        tmp2 =  self.pip2.fit_transform(X)
        col1 = list(tmp1.columns)
        col2 = list(tmp2.columns)
        cols = col1+col2
        aux = np.concatenate((tmp1.to_numpy(),tmp2.to_numpy()),axis=1)
        
        return pd.DataFrame(aux,columns=cols)
        

In [ ]:
preprocessing = Pipeline([
       ('outliers',RemoveOutliers()),
        ('preprocess',MergeFeatures(num_pipeline,cat_pipeline))
])

## 2. Data visualization
Visit the [dashboard](https://stroke-risk-factors.herokuapp.com) allowing you to interactively explore the stroke risk factors.
### 2.1 Distributions
In this section, I will compare the distribution of the variables in the two subsets 'Stroke' and "No Stroke'. If the distribution of a given feature matches in the two sets within statistical uncertainties, then we can conclude that the selected feature is not correlated with the target. The results obtained in this section will be compared in Sec. 4 with the findings of the dimensionality reduction.

Findings:
1. From the plot below it is clear that age is one of the most important features in the dataset, being the distribution among patients who were subjected to stroke shifted towards higher values. In particular, the distribution of the subset 'Stroke' is strongly peaked around 80 years. 
2. **BMI** may have an impact on the probability to be subjected to stroke since the distribution of the subset 'Stroke' shows a smaller variance compared to the 'No Stroke' one.
3. The distributions of the **average glucose level** show similar qualitative behavior in the two datasets. However, among the patients subjected to stroke is more common to have higher glucose levels (i.e. ~200).
4. **Gender** seems not to be a predominant feature here, since no significant differences between the two distributions have been observed.
5. **Hypertension** is a relevant feature, which is statistically more present among patients subjected to stroke.
6. **Heart disease**, like hypertension, can be relevant.
7. The feature **ever_married** shows a correlation with the target 'Stroke'. However, this can be explained from the correlation of this feature with the age, which we know is an important variable in this analysis. 
8. The **work type** shows a correlation with the target, which is however due to the presence of children in the sample. Thus, this feature is expected to be correlated with age.
9. The distributions of the **residence type** show a mild asymmetry between the two datasets. However, this can be due to statistical fluctuations.
10. Finally, the **smoking status** is correlated with the target. Indeed, there are fewer patients with 'unknown' smoking status in the dataset 'Stroke' in favor of the status 'formerly smoked'. 


In [ ]:
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2.5})
fig = plt.figure(figsize=(12,13))
gs = fig.add_gridspec(5, 2)
gs.update(wspace=0.1, hspace=0.65)
axes = []
for j in range(10):
    axes.append(fig.add_subplot(gs[int(j/2),j%2]))
    
#Feature 1: age
sns.kdeplot(data=df, x="age", hue="stroke",
   fill=True, common_norm=False, palette="rocket",
   alpha=.5, linewidth=0,ax=axes[0],cut=0,legend=False)
axes[0].legend(loc='upper left',title='Stroke', labels = ['Yes','No'])


#Feature 2: bmi
sns.kdeplot(data=df, x="bmi", hue="stroke",
   fill=True, common_norm=False, palette="rocket",
   alpha=.5, linewidth=0,ax=axes[1],legend=False,cut=0)




#Feature 3: bmi
sns.kdeplot(data=df, x="avg_glucose_level", hue="stroke",
   fill=True, common_norm=False, palette="rocket",
   alpha=.5, linewidth=0,ax=axes[2],legend=False,cut=0)


#Feature 4:gender
sns.histplot(y='gender', hue="stroke",
            palette="rocket",
            data=df, ax=axes[3],stat='density',
            common_norm=False,legend=False)

#Feature 5:hypertension
tmp_df = df.copy()
tmp_df['hypertension'] = tmp_df['hypertension'].map(lambda x: {0:'No',1:'Yes'}[x])

sns.histplot(y='hypertension', hue="stroke",
            palette="rocket",
            data=tmp_df, ax=axes[4],stat='density',
            common_norm=False,legend=False)


#Feature 6:heart_disease
tmp_df = df.copy()
tmp_df['heart_disease'] = tmp_df['heart_disease'].map(lambda x: {0:'No',1:'Yes'}[x])

sns.histplot(y='heart_disease', hue="stroke",
            palette="rocket",
            data=tmp_df, ax=axes[5],stat='density',
            common_norm=False,legend=False)

for j,feat in enumerate(['ever_married','work_type','Residence_type','smoking_status']):
    sns.histplot(y=feat, hue="stroke",
            palette="rocket",
            data=df, ax=axes[6+j],stat='density',
            common_norm=False,legend=False)


for j in range(10):
    which_ax = ['bottom','right'] if j%2==0 else ['bottom','left']
    for s in which_ax:
        axes[j].spines[s].set_visible(False)
    if(j%2==1):
        axes[j].yaxis.tick_right()
        axes[j].yaxis.set_label_position("right")
    axes[j].xaxis.tick_top()
    axes[j].xaxis.set_label_position("top")

### 2.2 Study of the correlation matrix

The dataset contains continuous and categorical variables. Moreover, the categorical variables can have from 2 (binary) possible values (Residence type) to 5 categories (Work type). Given these premises, we would need to define an inhomogeneous correlation matrix, by applying different correlation coefficients between different couples of variables. For instance, we could use the Pearson correlation coefficient with continuous-continuous variables, the point-biserial between continuos-binary variables and the [correlation ratio](https://en.wikipedia.org/wiki/Correlation_ratio) between categorical-categorical variables. However effective, this may have been misleading, since the comparison between the different correlation coefficients would be meaningless. Instead, I adopted a new method proposed in [this paper](https://www.sciencedirect.com/science/article/abs/pii/S0167947320301341) allowing to define a unique correlation coefficient for every couple of variables in the dataset. The method is implemented in the [Phi_K Correlation Analyzer Library](https://phik.readthedocs.io/en/latest/) in Python. The library can be easily installed through pip.

Below I show the correlation matrix between the features. 
* In the first place, the 'stroke' feature is correlated (in reverse order of importance) with age, heart disease,hypertension, average glucose level, marital status, smoking status, work type, BMI.
* However, as mentioned in [Sec. 2.1](#2.1-Distributions), the correlation with gender is negligible, as well as the one with the residence type. 
* Among the variables correlated with the target, many of them are strongly correlated with age. In particular the marital status, the work type, the bmi and smoking status have correlation coefficient greater than 0.3 with the age. For this reason it will be important to reduce the number of features.

In [ ]:
categ_feat = ['gender','Residence_type','work_type','ever_married','smoking_status']
encoder = OrdinalEncoder()
result = encoder.fit_transform(df[categ_feat])
df_ordinal = df.copy()
df_ordinal[categ_feat] = result
f,ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(df_ordinal.corr(), dtype=bool))
sns.heatmap(df_ordinal.phik_matrix(),mask=mask, annot=True, linewidths=.5, fmt= '.2f',ax=ax)
plt.show()

## 3. Testing resampling techniques with XGBoost binary classifier
The dataset is strongly imbalanced, just 5% of the patients were subjected to a stroke. For this reason, it is necessary to employ a **resampling** technique. All the algorithms employed here to tackle the issue are present in the [imblearn](https://imbalanced-learn.org/stable/) library. 
* In particular, I will employ some oversampling techniques (i.e. *SMOTE, Random oversampling*), one undersampling technique (i.e. Random undersampling) and some mixed algorithms (i.e. *SMOTE + ENN* and *SMOTE + Tomek links*).
* The resampling algorithms will be tested with the XGBoost binary classifier.
* All the resampling techniques will be employed **on the training set only**. IMPORTANT: this is automatically accomplished by using an imblearn pipeline object.
* To test the different model we will use the stratified k-fold cross-validation, that preserves the imbalanced class distribution in each fold. 

In [ ]:
def compute_score(y_train,y_test,y_train_p,y_test_p):
    test_ac = accuracy_score(y_test,y_test_p)
    train_ac = accuracy_score(y_train,y_train_p)
    test_f1 = f1_score(y_test,y_test_p)
    train_f1 = f1_score(y_train,y_train_p)

    cmte = confusion_matrix(y_test,y_test_p)
    
    cmtr = confusion_matrix(y_train,y_train_p)
        
    d = {'Dataset':['Test','Train'],'F1':[test_f1,train_f1]}
    d['Accuracy'] = [test_ac,train_ac]
    d['Recall'] = [recall_score(y_test,y_test_p),recall_score(y_train,y_train_p)]
    d['Precision'] = [precision_score(y_test,y_test_p),precision_score(y_train,y_train_p)]
    d['ROC AUC'] = [roc_auc_score(y_test,y_test_p),roc_auc_score(y_train,y_train_p)]
    d['False Positives (T-I)'] = [cmte[0,1],cmtr[0,1]]
    d['False Negatives (T-II)'] = [cmte[1,0],cmtr[1,0]]
    d['True Positives'] = [cmte[1,1],cmtr[1,1]]
    d['True Negatives'] = [cmte[0,0],cmtr[0,0]]
    
    
    
    display(pd.DataFrame(d).head(2))

In [ ]:
def plot_res(results):
    epochs = len(results['validation_0']['error'])
    x_axis = range(0, epochs)
    fig = plt.figure(figsize=(6,4))
    gs = fig.add_gridspec(1, 1)
    gs.update(wspace=0.1, hspace=0.65)
    ax = []
    ax.append(fig.add_subplot(gs[0,0]))
#     ax.append(fig.add_subplot(gs[0,1]))
#     ax[0].plot(x_axis, results['validation_0']['f1'], label='Train',color='red')
#     ax[0].plot(x_axis, results['validation_1']['f1'], label='Test',color='purple')
    
#     ax[0].legend()
#     ax[0].set_ylabel(r'$F_1$ score')
#     ax[0].set_title(r'XGBoost $F_1$')
    
    
    ax[0].plot(x_axis, results['validation_0']['error'], label='Train',color='red')
    ax[0].plot(x_axis, results['validation_1']['error'], label='Test',color='purple')
    ax[0].set_ylabel('Error')
    ax[0].set_title('XGBoost Error')
    
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['top'].set_visible(False)
#     ax[1].spines['left'].set_visible(False)
#     ax[1].spines['top'].set_visible(False)
    
#     ax[1].yaxis.tick_right()
#     ax[1].yaxis.set_label_position("right")
    
    plt.show()


def f1_metric(predt: np.ndarray, dtrain: xgb.DMatrix, threshold=0.5):
    '''F_1 score.'''
    y = dtrain.get_label()
    y_p = (predt > threshold).astype(int)
    return 'f1', f1_score(y,y_p)

In [ ]:
df_cols = list(preprocessing.fit_transform(df).columns)
df_cols.remove('stroke')


df_clean = preprocessing.fit_transform(df.drop(['id'],axis=1))
x_train, x_test, y_train, y_test = train_test_split(df_clean.drop(['stroke'],axis=1), df_clean['stroke'], 
                                                    test_size=0.25, random_state=42)


pars_xgb = {'objective':'binary:logistic',
            'random_state':7,
            'scale_pos_weight':1,
            'use_label_encoder':False,
            'eval_metric':'error',
            'learning_rate':0.01,
            'colsample_bytree': 0.05,
            'subsample': 0.8,
            'n_estimators':2000, 
            'reg_alpha': 0.3,
            'n_jobs':1,
            'max_depth':10, 
            'gamma':10
           }


def plot_roc(fpr,tpr,label=None):
    plt.figure(figsize=(5,4))
    plt.plot(fpr,tpr,linewidth=2,label=label)
    plt.plot([0,1],[0,1],'k--')
    plt.axis([0,1,0,1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

def test_resampling(resampling,x_train, x_test, y_train, y_test, cols = df_cols):
    
    pip_res = Pipeline([    
        ('rsp',resampling), 
            ('xgba',xgb.XGBClassifier(**pars_xgb))])
    
    eval_set = [(x_train,y_train)]
    pip_res = pip_res.fit(x_train,y_train,xgba__eval_metric=f1_metric, 
                         xgba__eval_set=eval_set, xgba__verbose=False)

    scores = pip_res.predict_proba(x_train)[:,1]
    
    fpr,tpr,thresholds = roc_curve(y_train,scores)
    plot_roc(fpr,tpr)
    
    scoring = ['accuracy', 'precision', 'recall', 'f1']
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    scores = cross_validate(pip_res, x_train,y_train, scoring=scoring, cv=cv, n_jobs=-1,  return_train_score=True)

    print('Cross-Validation: Accuracy: {:.2f}, Precison: {:.2f}, Recall: {:.2f}, F1: {:.2f}'.format(np.mean(scores['test_accuracy']), np.mean(scores['test_precision']), np.mean(scores['test_recall']), np.mean(scores['test_f1'])))

    
    compute_score(y_train,y_test,pip_res.predict(x_train),pip_res.predict(x_test))

#     results = pip_res[1].evals_result()
    
#     plot_res(results)

#### SMOTE

In [ ]:
resampling = SMOTE(random_state=18)
test_resampling(resampling,x_train, x_test, y_train, y_test)

#### SMOTE + ENN

In [ ]:
resampling = SMOTEENN(random_state=7)
test_resampling(resampling,x_train, x_test, y_train, y_test)

#### SMOTE + Tomek links

In [ ]:
resampling = SMOTETomek(random_state=7)
test_resampling(resampling,x_train, x_test, y_train, y_test)

#### Random oversampler

In [ ]:
resampling = RandomOverSampler(random_state=7)
test_resampling(resampling,x_train, x_test, y_train, y_test)

#### Random undersampler

In [ ]:
resampling = RandomUnderSampler(random_state=7)
test_resampling(resampling,x_train, x_test, y_train, y_test)

From the results above it is clear that all the algorithms (except for the undersampling one) show qualitatively the same performances. 
* Indeed, in all cases, the $F_1$ score, which is the appropriate metric for imbalanced binary datasets, is greater than 0.9 on the training set and ~0.25 on the test set. By tuning the hyperparameters of these classifiers it is possible to further increase the $F_1$ score on the training set, even to 0.99, but this would result in overfitting since no gain would be obtained on the $F_1$ score on the test set.
* In all cases, the number of false negatives is quite large compared to the number of true positives. This is bad, due to the relatively small number of patients subjected to stroke in the dataset.
* The undersampling algorithm gives a smaller $F_1$ score on the training set. 

### 3.1 Hyperparameters tuning with GridSearchCV

In [ ]:
def update_pars(new_p,pars):
    for p in new_p.keys():
        p_new = p.replace('xgba__','')
        pars[p_new] = new_p[p]

In [ ]:
def test_in_range(x_trainf, x_testf, y_trainf, y_testf, pars,param_test, resampling = SMOTEENN(random_state=7)):
    
    pip_res = Pipeline([('rsp',resampling), 
                              ('xgba',xgb.XGBClassifier(**pars_xgb))])
    
    new_params = [{'xgba__' + key: el[key] for key in el} for el in param_test]
    gsearch = GridSearchCV(estimator = pip_res, 
    param_grid = new_params, scoring='f1',n_jobs=-1, cv=5)

    gsearch.fit(x_trainf,y_trainf,xgba__verbose=False)
    
    
    scores = gsearch.predict_proba(x_trainf)[:,1]
    
    fpr,tpr,thresholds = roc_curve(y_trainf,scores)
    plot_roc(fpr,tpr)
    
    print('Best score: ',gsearch.best_params_)

    update_pars(gsearch.best_params_,pars_xgb)

    clf_rf = pip_res      
    clr_rf = pip_res.fit(x_trainf,y_trainf)

    compute_score(y_trainf,y_testf,clr_rf.predict(x_trainf),clr_rf.predict(x_testf))

In [ ]:
# resampling = RandomUnderSampler(random_state=7)
# x_tmp, y_tmp = resampling.fit_resample(x_train, y_train)
# x_train_res = pd.DataFrame(x_tmp,columns=df_cols)
# y_train_res = pd.Series(y_tmp)

param_test1 = [
    {'scale_pos_weight':range(1,10,3)},
    {'max_depth':range(3,15,2),'min_child_weight':range(1,6,2)},
    {'gamma':[i/10.0 for i in range(0,5)]},
    { 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]}
]

pars_xgb = {'objective':'binary:logistic',
            'random_state':7,
            'scale_pos_weight':1,
            'use_label_encoder':False,
            'eval_metric':'error',
            'learning_rate':0.01,
            'colsample_bytree': 0.05,
            'subsample': 0.8,
            'n_estimators':2000, 
            'reg_alpha': 0.3,
            'n_jobs':1,
            'max_depth':10, 
            'gamma':10
           }

test_in_range(x_train, x_test, y_train, y_test, pars_xgb, param_test1)

## 4. Dimensionality reduction

#### 4.A Univariate analysis
Univariate tests measure the correlation between the selected feature and the target. Each feature is analyzed independently, not taking into account the correlation between different features.    

In [ ]:
# select_feature = SelectKBest(chi2, k=5).fit(x_train, y_train)
# order_ind = select_feature.scores_.argsort()
# print('Score list:', select_feature.scores_[order_ind[::-1]])
# print('Feature list:', x_train.columns[order_ind[::-1]])

The score list above confirms the findings of the [data visualization section](#2.-Data-visualization). In particular, age is the most important feature to consider here, together with the average glucose level. Among the other important variables selected by the univariate algorithm it is interesting to mention the  'ever_married_Yes', that we know is strongly correlated with age, as well as 'work_type_children'. For this reason, before proceeding with the dimensionality reduction it is necessary to employ a multivariate algorithm.

### 4.B Recursive feature elimination (RFE) with random forest

Random forest can be employed in combination with the [feature ranking with recursive feature elimination](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) algorithm. Due to the strong correlation between features, the expected outcome significantly differs from the output of the univariate algorithm employed in the previous section.

In [ ]:
resampling = SMOTEENN(random_state=7)
# x_tmp, y_tmp = resampling.fit_resample(x_train, y_train)
# x_train_tmp = pd.DataFrame(x_tmp,columns=df_cols)
# y_train_tmp = pd.Series(y_tmp)

clf_rf = RandomForestClassifier()      
rfe = RFE(estimator=clf_rf, n_features_to_select=5, step=1)
pip_res = Pipeline([('rsp',resampling), 
                              ('rf',rfe)])
rfe = rfe.fit(x_train, y_train)
print('Selected features:',x_train.columns[rfe.support_])

As expected, the best features selected by the algorithm do not include the varialbe 'ever_married_Yes', which is strongly correlated with age. Among the five best features the variables 'smoking_status_never smoked' and 'smoking_status_smokes' have been included, which however have a weak correlation with the target. These variables may not give significative additional information to the classifier. Let's test now the XGBoost classifier with SMOTEENN employing the selected features...

In [ ]:
pars_xgb = {'objective':'binary:logistic',
            'random_state':7,
            'scale_pos_weight':1,
            'use_label_encoder':False,
            'eval_metric':'error',
            'learning_rate':0.01,
            'colsample_bytree': 0.05,
            'subsample': 0.8,
            'n_estimators':2000, 
            'reg_alpha': 0.3,
            'n_jobs':1,
            'max_depth':10, 
            'gamma':10
           }

rf_rfe_columns = list(x_train.columns[rfe.support_])
test_resampling(resampling,x_train[rf_rfe_columns], 
                x_test[rf_rfe_columns], y_train, y_test, rf_rfe_columns)

The $F_1$ score on the test set has slightly improved compared to the classification performed including all the available features in [Sec. 3](#3.-Testing-resampling-techniques-with-XGBoost-binary-classifier). However, the number of features has been drastically reduced to 5. In the next section, I am going to test another multivariate feature selection algorithm and compare the results with what obtained here.

### 4.C RFE with Cross Validation (RFECV) with random forest
Below, I am going to the RFE algorithm with cross-validation, employing the random forest classifier. This will allow to crosscheck the best features selected in the previous subsection and to get the optimal number of features.

In [ ]:
from sklearn.feature_selection import RFECV

clf_rf = RandomForestClassifier() 
rfecv = RFECV(estimator=clf_rf, step=1, cv=10,scoring='f1')
rfecv = rfecv.fit(x_train, y_train)

print('Optimal number of features :', rfecv.n_features_)
print('Best features :', x_train.columns[rfecv.support_])

The optimal number of features is 3, and they correspond with the ones selected in the previous subsection.

In [ ]:
rf_rfecv_columns = list(x_train.columns[rfecv.support_])
test_resampling(resampling,x_train[rf_rfecv_columns], 
                x_test[rf_rfecv_columns], y_train, y_test, rf_rfecv_columns)

The $F_1$ score on the test set remains constant compared to the results of [Sec. 4.C](#4.C-Recursive-feature-elimination-(RFE)-with-random-forest).

## 4.D Factor Analysis of Mixed Data (FAMD) for feature extraction

Principal component analysis (PCA) is by far the most common dimensionality reduction techniques, but can be employed with datasets containing continuous variable only. In this case, the dataset contains instead a combination of continuous and categorical variables. For this reason, [factor analysis of mixed data (FAMD)](https://en.wikipedia.org/wiki/Factor_analysis_of_mixed_data) is the technique most suitable to perform feature extraction on the stroke prediction dataset. It can be viewed as the combination of PCA for continuous variables and multiple correspondence analysis (MCA) for categorical variables.  
FAMD is not available in scikit-learn, while in R it can be found in the [PCAmix](https://cran.r-project.org/web/packages/PCAmixdata/index.html) package. In Python, it was implemented in a privately maintained package, [Prince](https://github.com/MaxHalford/prince/tree/988f7fe01b6e4c9476517d1939f5fe0e13deb158).



![r-blogger](https://i0.wp.com/www.sthda.com/sthda/RDoc/images/multivariate-analysis-factoextra.png?w=578)  

Figure taken from [r-blogger](https://www.r-bloggers.com/2017/02/factoextra-r-package-easy-multivariate-data-analyses-and-elegant-visualization/) website.

In [ ]:
df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

tmp = Pipeline([
       ('outliers',RemoveOutliers()),
        ('preprocess',MergeFeatures(num_pipeline,DataFrameSelector(categ_feat)))
]).fit_transform(df.drop(['id'],axis=1))

tmp_dic = {}
for c in categ_feat:
    tmp_dic[c]='str'
for c in num_feat:
    tmp_dic[c]=float
tmp = tmp.astype(tmp_dic)


famd_stroke = prince.FAMD(
     n_components=16,
     n_iter=10,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42)
principalComponents_stroke = famd_stroke.fit_transform(tmp)

In the plot below we see the distribution of the "inertia" (i.e. variance in the nomenclature of the package Prince), among the components selected by the FAMD algorithm. The first component carries 20% of the variance, the remaining ~80% of the variance is distributed among 13 components. For instance, 80% of the variance is contained in the first 8 components.

In [ ]:
sns.set()
plt.figure(1, figsize=(10, 9))
sns.lineplot(data=np.cumsum(famd_stroke.explained_inertia_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

plt.bar(range(0,16), famd_stroke.explained_inertia_,
        alpha=0.5,
        align='center')
plt.step(range(0,16), np.cumsum(famd_stroke.explained_inertia_),
         where='mid',
         color='red')

In [ ]:
n_pca = 5
pca_cols = []
for j in range(n_pca):
    pca_cols.append('pca%d'%(j))
principalComp_stroke_df = pd.DataFrame(data = np.array(principalComponents_stroke)[:,:n_pca],
                                      columns=pca_cols)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(principalComp_stroke_df, df_clean['stroke'], 
                                                    test_size=0.25, random_state=42)

In [ ]:
test_resampling(resampling,x_train, 
                x_test, y_train, y_test, pca_cols)

## Neural network modeldef plot_cm(labels, predictions, p=0.5):
  


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from keras.regularizers import l2
print(tf.__version__)

In [ ]:
df_clean.drop('stroke',axis=1).head()

In [ ]:
resampling = SMOTE(random_state=7)

x_train, x_test, y_train, y_test = train_test_split(df_clean.drop('stroke',axis=1),df_clean['stroke'], 
                                                    test_size=0.25, random_state=42)

x_tmp, y_tmp = resampling.fit_resample(x_train, y_train)
x_train_tmp = pd.DataFrame(x_tmp,columns=df_cols)
y_train_tmp = pd.Series(y_tmp)


model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(len(df_clean.columns)-1,)),
    tf.keras.layers.Dense(20, activation="relu",kernel_regularizer=l2(0.01)),
#     tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])


lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-3 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.Adam(lr=0.5*1e-3)

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=optimizer,
              metrics=[METRICS])

print(len(df_clean.columns))

counts = np.bincount(y_train)
weight_for_0 = (1.0 / counts[0])
weight_for_1 = (1.0 / counts[1])
class_weight = {0: weight_for_0, 1: weight_for_1}
class_weight = {0: 1, 1: 1}
history = model.fit(x_train_tmp,y_train_tmp,epochs=150,verbose=0,#, callbacks=[lr_schedule],
                       validation_data=(x_test, y_test),
                        class_weight=class_weight)

In [ ]:
def plot_loss(history, label, n):
    plt.semilogy(history.epoch, history.history['loss'], label='Train ' + label)
    plt.semilogy(history.epoch, history.history['val_loss'], label='Val ' + label,
               linestyle="--")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()


In [ ]:
plot_loss(history, "set", 0)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

In [ ]:
treshold = 0.5
out = model.predict(x_test)
test_pred = np.array([0 if p < treshold else 1 for p in out])
out = model.predict(x_train_tmp)
train_pred = np.array([0 if p < treshold else 1 for p in out])
plot_cm(y_test, test_pred)
compute_score(y_train_tmp,y_test,train_pred,test_pred)